In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
#Get CSV from WeatherPy
cities_df = pd.read_csv("../WeatherPy/Output_Data/cities.csv")

### Humidity Heatmap

In [14]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

# Customize the size of the map
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [4]:
ideal_df = cities_df.loc[(cities_df["Max Temp"] > 60) & (cities_df["Max Temp"] < 85) & \
                         (cities_df["Humidity"] < 40) & (cities_df["Wind Speed"] < 10) & \
                         (cities_df["Cloudiness"] < 50), :]

mapcities_df = ideal_df.dropna()
mapcities_df = mapcities_df.reset_index(drop=True)

### Hotel Map

In [5]:
#Set up hotel dataframe
hotel_df = mapcities_df.loc[:,["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""


In [6]:
#API request to get hotel names
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    location = f"{row['Lat']},{row['Lng']}"
    city = row["City"]

    # add keyword to params dict
    params["location"] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        
    print("------------")


Retrieving Results for Index 0: Neyshabur.
Closest hotel is Amiran Hotel.
------------
Retrieving Results for Index 1: Mount Isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 2: Tursunzoda.
Closest hotel is Stoyanka Charli.
------------
Retrieving Results for Index 3: Narsaq.
Closest hotel is Hotel Narsaq.
------------
Retrieving Results for Index 4: Arrondissement de Foix.
Closest hotel is Hôtel TerraNostra - Le Bellevue.
------------
Retrieving Results for Index 5: Arrondissement de Narbonne.
Closest hotel is Bed & Breakfast Demeure de Roquelongue Saint-André de Roquelongue.
------------
Retrieving Results for Index 6: Jauja.
Closest hotel is Hotel TunanMarca.
------------
Retrieving Results for Index 7: San Rafael.
Closest hotel is Hotel San Rafael.
------------
Retrieving Results for Index 8: Kaniama.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: Española.
Closest hotel is Santa Claran Hotel.
------------


In [7]:
#Erase city with no result
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index(drop=True)

hotel_df

,City,Lat,Lng,Country,Hotel Name
0,Neyshabur,36.21,58.80,IR,Amiran Hotel
1,Mount Isa,-20.73,139.50,AU,ibis Styles Mt Isa Verona
2,Tursunzoda,38.51,68.23,TJ,Stoyanka Charli
3,Narsaq,60.92,-46.05,GL,Hotel Narsaq
4,Arrondissement de Foix,42.83,1.58,FR,Hôtel TerraNostra - Le Bellevue
5,Arrondissement de Narbonne,43.08,2.83,FR,Bed & Breakfast Demeure de Roquelongue Saint-A...
6,Jauja,-11.78,-75.50,PE,Hotel TunanMarca
7,San Rafael,-34.62,-68.33,AR,Hotel San Rafael
8,Española,35.99,-106.08,US,Santa Claran Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_locations,
        info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…